In [3]:
import pandas as pd
from datetime import datetime
import gurobipy as gp
from gurobipy import GRB


def run_nba_schedule_project():
    """
    This function loads the NBA schedule data, performs the analysis from
    Part 1, and solves the Integer Programs for Parts 2 and 3 using Gurobi.
    """

    print("--- Starting NBA Schedule Project (Gurobi Version) ---")

    # =========================================================================
    # --- Part 1: Data Analysis ---
    # =========================================================================
    print("\n--- Part 1: Schedule Statistics ---")

    df = pd.read_csv('games.csv')

    # Clean data
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Visitor'] = df['Visitor'].str.strip()
    df['Home'] = df['Home'].str.strip()

    # Get unique teams and sorted dates
    all_teams = sorted(list(pd.concat([df['Home'], df['Visitor']]).unique()))
    all_dates = sorted(list(df['Date'].unique()))

    print(f"Loaded {len(df)} games.")
    print(f"Found {len(all_teams)} teams.")
    print(f"Found {len(all_dates)} unique dates.")

    home_dates = {}
    away_dates = {}
    home_matchups = {}
    away_matchups = {}
    team_game_dates = {} # For Part 3

    for team in all_teams:
        # (a) all dates when team i played home
        home_dates[team] = sorted(list(df[df['Home'] == team]['Date'].unique()))

        # (d) all dates when team j played away (using 'team' as 'j')
        away_dates[team] = sorted(list(df[df['Visitor'] == team]['Date'].unique()))

        # Store all game dates (home or away) for Part 3
        all_games_for_team = sorted(list(set(home_dates[team] + away_dates[team])))
        team_game_dates[team] = all_games_for_team

        # (b) for each team j, a number of times team i played against team j at home
        home_matchups[team] = df[df['Home'] == team]['Visitor'].value_counts().to_dict()

        # (c) for each team j, the number of times team i played against team j away
        away_matchups[team] = df[df['Visitor'] == team]['Home'].value_counts().to_dict()

        # Print the information
        print(f"\n--- Team: {team} ---")
        print(f"  (a) Home Dates: {[d.strftime('%Y-%m-%d') for d in home_dates[team]]}")
        print(f"  (d) Away Dates: {[d.strftime('%Y-%m-%d') for d in away_dates[team]]}")
        print(f"  (b) Home Matchups (vs Visitor): {home_matchups[team]}")
        print(f"  (c) Away Matchups (vs Home): {away_matchups[team]}")

    # --- End of Part 1 ---

    # Gurobi variable keys: (home_team, away_team, date)
    var_keys = [(i, j, d) for i in all_teams for j in all_teams if i != j for d in all_dates]

    # =========================================================================
    # --- Part 2: Integer Program (Feasibility) ---
    # =========================================================================
    print("\n--- Part 2: Integer Program (Feasibility) ---")

    # Suppress Gurobi license output to keep console clean
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()

        # Create the model
        m_feasible = gp.Model("NBASchedule_Feasibility", env=env)

        # --- Decision Variables ---
        # X_i_j_d = 1 if team i (home) plays team j (visitor) on date d
        X = m_feasible.addVars(var_keys, vtype=GRB.BINARY, name="X")

        # --- Objective Function (Dummy) ---
        m_feasible.setObjective(0, GRB.MINIMIZE)

        # --- Constraints ---
        print("Adding feasibility constraints (e, f, g, h)...")

        m_feasible.addConstrs(
            (X.sum(i, '*', d) == (1 if d in home_dates[i] else 0)
             for i in all_teams for d in all_dates), name="HomeGame"
        )
        m_feasible.addConstrs(
            (X.sum('*', i, d) == (1 if d in away_dates[i] else 0)
             for i in all_teams for d in all_dates), name="AwayGame"
        )
        m_feasible.addConstrs(
            (X.sum(i, j, '*') == home_matchups[i].get(j, 0)
             for i in all_teams for j in all_teams if i != j), name="HomeCount"
        )
        m_feasible.addConstrs(
            (X.sum(j, i, '*') == away_matchups[i].get(j, 0)
             for i in all_teams for j in all_teams if i != j), name="AwayCount"
        )

        # --- Solve Part 2 ---
        print("Solving Part 2 (Feasibility)...")
        m_feasible.optimize()

        # --- Part 2 Deliverable (Save CSV) ---
        if m_feasible.Status == GRB.OPTIMAL:
            print("Part 2 Status: Feasible schedule found.")

            # Extract the feasible schedule
            schedule_rows_p2 = []
            for (i, j, d) in var_keys:
                if X[i, j, d].X > 0.5: # Use .X to get variable value
                    schedule_rows_p2.append({
                        'date': d,
                        'team playing home': i,
                        'team playing away': j
                    })

            schedule_df_p2 = pd.DataFrame(schedule_rows_p2).sort_values(by='date')
            schedule_df_p2['date'] = schedule_df_p2['date'].dt.strftime('%Y-%m-%d')

            # Save to CSV file
            schedule_df_p2.to_csv("feasible_schedule_part2.csv", index=False)
            print("Part 2: Feasible schedule saved to 'feasible_schedule_part2.csv'")

        elif m_feasible.Status == GRB.INFEASIBLE:
            print("Part 2 Status: Infeasible. No schedule matches the constraints.")
            return # Stop if Part 2 is infeasible
        else:
            print(f"Part 2 Status: {m_feasible.Status}")
            return
        # --- **END NEW BLOCK** ---

    # --- End of Part 2 ---

    # =========================================================================
    # --- Part 3: IP with Time Zone Constraint ---
    # =========================================================================
    print("\n--- Part 3: IP with Time Zone Constraint ---")

    # Time Zone assumptions (ET=0, CT=-1, MT=-2, PT=-3)
    team_to_tz = {
        'Atlanta Hawks': 0, 'Boston Celtics': 0, 'Brooklyn Nets': 0,
        'Cleveland Cavaliers': 0, 'Miami Heat': 0, 'New York Knicks': 0,
        'Philadelphia 76ers': 0, 'Toronto Raptors': 0,
        'Chicago Bulls': -1, 'Dallas Mavericks': -1, 'Houston Rockets': -1,
        'Milwaukee Bucks': -1,
        'Denver Nuggets': -2, 'Phoenix Suns': -2,
        'Golden State Warriors': -3, 'Los Angeles Lakers': -3
    }

    # Check if all teams from data are in our TZ map
    for team in all_teams:
        if team not in team_to_tz:
            print(f"Warning: Team '{team}' not found in Time Zone map. Assigning ET (0).")
            team_to_tz[team] = 0

    # Suppress Gurobi license output
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()

        # Create the new problem
        m_tz = gp.Model("NBASchedule_TZ", env=env)

        # --- Decision Variables ---
        X_tz = m_tz.addVars(var_keys, vtype=GRB.BINARY, name="X")

        # --- Objective Function (Dummy) ---
        m_tz.setObjective(0, GRB.MINIMIZE)

        # --- Add All Constraints from Part 2 ---
        print("Adding feasibility constraints (e, f, g, h) to Part 3 model...")
        m_tz.addConstrs(
            (X_tz.sum(i, '*', d) == (1 if d in home_dates[i] else 0)
             for i in all_teams for d in all_dates), name="HomeGame"
        )
        m_tz.addConstrs(
            (X_tz.sum('*', i, d) == (1 if d in away_dates[i] else 0)
             for i in all_teams for d in all_dates), name="AwayGame"
        )
        m_tz.addConstrs(
            (X_tz.sum(i, j, '*') == home_matchups[i].get(j, 0)
             for i in all_teams for j in all_teams if i != j), name="HomeCount"
        )
        m_tz.addConstrs(
            (X_tz.sum(j, i, '*') == away_matchups[i].get(j, 0)
             for i in all_teams for j in all_teams if i != j), name="AwayCount"
        )

        # --- Add New Time Zone Constraints ---
        print("Adding Time Zone constraints...")

        # **Constraint is |tz2-tz1| + |tz3-tz2| <= 3 (from fact sheet)

        for i in all_teams:
            games_list = team_game_dates[i] # Sorted list of all dates for team i

            # Iterate through all 3-game sequences
            for g in range(len(games_list) - 2):
                d1, d2, d3 = games_list[g], games_list[g+1], games_list[g+2]

                # L_i_d = Time zone of the arena where team i plays on date d
                tz_d1 = gp.quicksum(X_tz[i, j, d1] * team_to_tz[i] for j in all_teams if i != j) + \
                        gp.quicksum(X_tz[j, i, d1] * team_to_tz[j] for j in all_teams if i != j)

                tz_d2 = gp.quicksum(X_tz[i, j, d2] * team_to_tz[i] for j in all_teams if i != j) + \
                        gp.quicksum(X_tz[j, i, d2] * team_to_tz[j] for j in all_teams if i != j)

                tz_d3 = gp.quicksum(X_tz[i, j, d3] * team_to_tz[i] for j in all_teams if i != j) + \
                        gp.quicksum(X_tz[j, i, d3] * team_to_tz[j] for j in all_teams if i != j)

                # We need to enforce |tz_d2 - tz_d1| + |tz_d3 - tz_d2| <= 3

                # Helper variables
                diff12 = m_tz.addVar(lb=-GRB.INFINITY, name=f"diff12_{i}_{g}")
                diff23 = m_tz.addVar(lb=-GRB.INFINITY, name=f"diff23_{i}_{g}")
                abs_diff12 = m_tz.addVar(name=f"abs_diff12_{i}_{g}")
                abs_diff23 = m_tz.addVar(name=f"abs_diff23_{i}_{g}")

                # Link helper variables
                m_tz.addConstr(diff12 == tz_d2 - tz_d1, name=f"link_diff12_{i}_{g}")
                m_tz.addConstr(diff23 == tz_d3 - tz_d2, name=f"link_diff23_{i}_{g}")

                # Gurobi's clean way to model absolute value
                m_tz.addGenConstrAbs(abs_diff12, diff12, name=f"gc_abs12_{i}_{g}")
                m_tz.addGenConstrAbs(abs_diff23, diff23, name=f"gc_abs23_{i}_{g}")

                # The final time zone constraint
                m_tz.addConstr(abs_diff12 + abs_diff23 <= 3, name=f"TZ_Sum_{i}_{g}")

        # --- Solve Part 3 ---
        print("Solving Part 3 (Time Zone Constraints)...")
        m_tz.optimize()

        # --- Part 3 Deliverables (Save CSV if optimal) ---
        if m_tz.Status == GRB.OPTIMAL:
            print("Part 3 Status: Feasible schedule with Time Zone constraints FOUND.")

            # Extract schedule
            schedule_rows_p3 = []
            for (i, j, d) in var_keys:
                if X_tz[i, j, d].X > 0.5: # Use .X to get variable value
                    schedule_rows_p3.append({
                        'date': d,
                        'team playing home': i,
                        'team playing away': j
                    })

            schedule_df_p3 = pd.DataFrame(schedule_rows_p3).sort_values(by='date')
            schedule_df_p3['date'] = schedule_df_p3['date'].dt.strftime('%Y-%m-%d')

            # Save to CSV file
            schedule_df_p3.to_csv("feasible_schedule_part3.csv", index=False)
            print("Part 3: Feasible schedule saved to 'feasible_schedule_part3.csv'")
            print(schedule_df_p3.head())

        elif m_tz.Status == GRB.INFEASIBLE:
            print("Part 3 Status: Infeasible.")
            print("NO FEASIBLE SCHEDULE exists that satisfies the time zone constraint.")

        else:
            print(f"Part 3: Solver finished with status: {m_tz.Status}")

    # --- End of Part 3 ---

# --- Run the Main Project Function ---
if __name__ == "__main__":
    run_nba_schedule_project()

--- Starting NBA Schedule Project (Gurobi Version) ---

--- Part 1: Schedule Statistics ---
Loaded 128 games.
Found 16 teams.
Found 16 unique dates.

--- Team: Atlanta Hawks ---
  (a) Home Dates: ['2025-11-03', '2025-11-07', '2025-11-15', '2025-11-17', '2025-11-19', '2025-11-23', '2025-11-27', '2025-11-28', '2025-11-29', '2025-12-25']
  (d) Away Dates: ['2025-11-01', '2025-11-05', '2025-11-11', '2025-11-13', '2025-11-21', '2025-12-01']
  (b) Home Matchups (vs Visitor): {'Toronto Raptors': 1, 'Golden State Warriors': 1, 'Boston Celtics': 1, 'Milwaukee Bucks': 1, 'Miami Heat': 1, 'New York Knicks': 1, 'Los Angeles Lakers': 1, 'Phoenix Suns': 1, 'Dallas Mavericks': 1, 'Chicago Bulls': 1}
  (c) Away Matchups (vs Home): {'Chicago Bulls': 1, 'Brooklyn Nets': 1, 'Denver Nuggets': 1, 'Houston Rockets': 1, 'Philadelphia 76ers': 1, 'Cleveland Cavaliers': 1}

--- Team: Boston Celtics ---
  (a) Home Dates: ['2025-11-01', '2025-11-07', '2025-11-13', '2025-11-19', '2025-11-23', '2025-11-28']
  (d) A